### Import Packages

In [2]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from nltk.corpus import stopwords
import string
from nltk import word_tokenize,ne_chunk,pos_tag,wordpunct_tokenize
import rake_nltk
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer,PorterStemmer
from itertools import chain, groupby, product
from collections import defaultdict, Counter
import networkx as nx
from networkx.algorithms import community
from gensim.models import Word2Vec,KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.ldamodel import LdaModel
import gensim
import statsmodels.api as sm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_ind
from scipy.stats import normaltest
% matplotlib notebook

//anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load Data

In [3]:
all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
current_data_folder_name = '(40000+)_by_year'
current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
data = pd.read_csv(os.path.join(current_data_store_path,'wholeFile_with_wholeTopic_v2_2.csv'))
topic_map_trend = pd.read_csv(os.path.join(current_data_store_path,'topic_map_trend_v2_2.csv'))

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,6,7,10,40,48,50,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
current_data_folder_name = 'external data/Journal Citation Report (Impact Factor)'
current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
JCR_all = pd.read_csv(os.path.join(current_data_store_path,'JCR_all.csv'))

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
JCR_all.head()

,Rank,Full Journal Title,JCR Abbreviated Title,ISSN,Total Cites,Journal Impact Factor,5-Year Impact Factor,Eigenfactor Score,Article Influence Score,Average Journal Impact Factor Percentile,year
0,1,ANNUAL REVIEW OF IMMUNOLOGY,ANNU REV IMMUNOL,0732-0582,"12,584",50.340,Not Available,Not Available,Not Available,99.569,2000
1,2,ANNUAL REVIEW OF BIOCHEMISTRY,ANNU REV BIOCHEM,0066-4154,"16,436",43.429,Not Available,Not Available,Not Available,99.839,2000
2,3,CELL,CELL,0092-8674,"153,747",32.440,Not Available,Not Available,Not Available,99.588,2000
3,4,NATURE GENETICS,NAT GENET,1061-4036,"38,840",30.910,Not Available,Not Available,Not Available,99.561,2000
4,5,NEW ENGLAND JOURNAL OF MEDICINE,NEW ENGL J MED,0028-4793,"135,613",29.512,Not Available,Not Available,Not Available,99.524,2000


In [6]:
emerging_topics = topic_map_trend[topic_map_trend['emerging']==1]
shrinking_topics = topic_map_trend[topic_map_trend['shrinking']==1]
neutral_topics = topic_map_trend[topic_map_trend['neutral']==1]

In [7]:
shrinking_topics.head()

,topic,doc_index,1995,1996,1997,1998,1999,2000,2001,2002,...,linear_coef,reach_max_recent,reach_min_recent,emerging,shrinking,emerging_period_start,emerging_period_end,shrinking_period_start,shrinking_period_end,neutral
0,polyester,105;125;274;326;392;549;560;582;652;692;1010;1...,0.003333,0.008403,0.000000,0.002193,0.008016,0.007313,0.010000,0.010974,...,-0.024182,NaN,1.0,NaN,1.0,NaN,NaN,2005.0,2009.0,NaN
11,titanium,26;49;117;139;154;173;251;262;303;313;316;347;...,0.023333,0.016807,0.026247,0.037281,0.030060,0.038391,0.050000,0.050754,...,-0.096000,NaN,1.0,NaN,1.0,NaN,NaN,2001.0,2005.0,NaN
16,study,28;41;52;87;102;125;145;154;169;194;209;232;24...,0.076667,0.072829,0.055118,0.059211,0.064128,0.071298,0.058333,0.060357,...,-0.126067,NaN,1.0,NaN,1.0,NaN,NaN,2000.0,2004.0,NaN
19,surface modification,23;107;133;148;208;214;261;291;350;359;426;469...,0.006667,0.030812,0.018373,0.021930,0.020040,0.018282,0.028333,0.012346,...,-0.051428,NaN,1.0,NaN,1.0,NaN,NaN,2007.0,2011.0,NaN
21,biocompatibility,5;6;11;18;26;41;102;105;143;158;232;237;265;27...,0.043333,0.053221,0.060367,0.072368,0.060120,0.060329,0.058333,0.050754,...,-0.100520,NaN,1.0,NaN,1.0,NaN,NaN,2013.0,2017.0,NaN


In [8]:
emerging_topics.head()

,topic,doc_index,1995,1996,1997,1998,1999,2000,2001,2002,...,linear_coef,reach_max_recent,reach_min_recent,emerging,shrinking,emerging_period_start,emerging_period_end,shrinking_period_start,shrinking_period_end,neutral
5,bone,3;9;10;19;37;49;55;58;60;63;64;65;85;87;93;94;...,0.090000,0.061625,0.094488,0.074561,0.090180,0.074954,0.090000,0.063100,...,0.148988,1.0,NaN,1.0,NaN,2002.0,2006.0,NaN,NaN,NaN
7,silk fibroin,591;794;847;848;864;1077;1145;1177;1199;1227;1...,0.000000,0.000000,0.000000,0.002193,0.000000,0.001828,0.000000,0.005487,...,0.126377,1.0,NaN,1.0,NaN,2006.0,2010.0,NaN,NaN,NaN
13,mesenchymal stem cell,215;526;720;849;861;871;975;1129;1131;1184;118...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001828,0.001667,0.002743,...,0.145362,1.0,NaN,1.0,NaN,2005.0,2009.0,NaN,NaN,NaN
15,biodegradation,43;361;595;600;670;1412;1459;1464;1489;1496;15...,0.003333,0.005602,0.000000,0.006579,0.002004,0.007313,0.003333,0.008230,...,0.012541,1.0,NaN,1.0,NaN,2008.0,2012.0,NaN,NaN,NaN
17,nanofibers,35;879;887;1001;1047;1141;1147;1160;1196;1328;...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001667,0.001372,...,0.109432,1.0,NaN,1.0,NaN,2004.0,2008.0,NaN,NaN,NaN


In [9]:
neutral_topics.head()

,topic,doc_index,1995,1996,1997,1998,1999,2000,2001,2002,...,linear_coef,reach_max_recent,reach_min_recent,emerging,shrinking,emerging_period_start,emerging_period_end,shrinking_period_start,shrinking_period_end,neutral
1,investigation,302;432;656;659;777;802;827;1038;1148;1626;174...,0.006667,0.002801,0.005249,0.006579,0.004008,0.005484,0.003333,0.009602,...,0.001517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,biofilm,147;159;231;272;611;678;712;837;896;1035;1144;...,0.003333,0.005602,0.005249,0.004386,0.004008,0.005484,0.006667,0.000000,...,0.018618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,self-assembly,35;163;292;460;480;523;541;545;594;597;706;719...,0.006667,0.000000,0.000000,0.013158,0.002004,0.003656,0.008333,0.005487,...,0.104989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,gel,1;149;501;596;702;727;999;1033;1066;1106;1116;...,0.006667,0.008403,0.007874,0.010965,0.008016,0.012797,0.003333,0.010974,...,0.023970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,mineralization,35;88;609;856;1203;1254;1281;1586;1699;1860;19...,0.003333,0.008403,0.005249,0.000000,0.006012,0.010969,0.005000,0.000000,...,-0.005089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### Assign Impact Factors to Original data

In [10]:
data_with_JIF = pd.merge(data,JCR_all[['year','ISSN','Rank','Journal Impact Factor','Total Cites','Average Journal Impact Factor Percentile','Full Journal Title']],how = 'left',left_on=['PY','SN'],right_on=['year','ISSN'])

In [11]:
data_with_JIF['Journal Impact Factor'].replace({'Not Available':np.NaN},inplace = True)

In [12]:
data_with_JIF['Journal Impact Factor'] = data_with_JIF['Journal Impact Factor'].apply(float)

In [13]:
data_with_JIF.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,emerging,shrinking,emergingAndShrinking,year,ISSN,Rank,Journal Impact Factor,Total Cites,Average Journal Impact Factor Percentile,Full Journal Title
0,J,"Takahashi, Y; Ohkawa, K; Ando, M; Yamada, M; Y...",NaN,NaN,NaN,"Takahashi, Y; Ohkawa, K; Ando, M; Yamada, M; Y...",NaN,NaN,Adsorption of endocrine disruptors and related...,MACROMOLECULAR MATERIALS AND ENGINEERING,...,NaN,1.0,NaN,2001.0,1438-7492,3346.0,0.835,89,63.076,MACROMOLECULAR MATERIALS AND ENGINEERING
1,J,"Granja, PL; Pouysegu, L; Petraud, M; De Jeso, ...",NaN,NaN,NaN,"Granja, PL; Pouysegu, L; Petraud, M; De Jeso, ...",NaN,NaN,Cellulose phosphates as biomaterials. I. Synth...,JOURNAL OF APPLIED POLYMER SCIENCE,...,1.0,NaN,NaN,2001.0,0021-8995,2868.0,0.992,"17,128",73.288,JOURNAL OF APPLIED POLYMER SCIENCE
2,J,"Granja, PL; Pouysegu, L; Deffieux, D; Daude, G...",NaN,NaN,NaN,"Granja, PL; Pouysegu, L; Deffieux, D; Daude, G...",NaN,NaN,Cellulose phosphates as biomaterials. II. Surf...,JOURNAL OF APPLIED POLYMER SCIENCE,...,1.0,1.0,1.0,2001.0,0021-8995,2868.0,0.992,"17,128",73.288,JOURNAL OF APPLIED POLYMER SCIENCE
3,J,"Jacoby, M",NaN,NaN,NaN,"Jacoby, M",NaN,NaN,Biomaterials - Tough bones,CHEMICAL & ENGINEERING NEWS,...,1.0,NaN,NaN,2001.0,0009-2347,4524.0,0.564,"1,459",54.065,CHEMICAL & ENGINEERING NEWS
4,J,"Arwin, H",NaN,NaN,NaN,"Arwin, H",NaN,NaN,Spectroscopic ellipsometry for characterizatio...,PHYSICA STATUS SOLIDI A-APPLIED RESEARCH,...,NaN,NaN,NaN,2001.0,0031-8965,2783.0,1.025,"6,010",57.273,PHYSICA STATUS SOLIDI A-APPLIED RESEARCH


### Impact Factor Analysis (T-tests)

In [14]:
emerging_topic_list = emerging_topics['topic'].values
shrinking_topic_list = shrinking_topics['topic'].values
neutral_topic_list = neutral_topics['topic'].values

In [15]:
def Jif_before_changing(topic,time_gap = 1,time_window = 1):
    #plt.figure(figsize = (10,6))
    if topic in list(emerging_topics['topic']):
        changing_point = emerging_topics.loc[emerging_topics['topic']==topic,'emerging_period_start'].values[0]
    if topic in list(shrinking_topics['topic']):
        changing_point = shrinking_topics.loc[shrinking_topics['topic']==topic,'shrinking_period_start'].values[0]
    if time_window > 1:
        return
    check_point = changing_point-time_gap
    sample = data_with_JIF[(data_with_JIF[topic].notnull())&(data_with_JIF['PY']==check_point)]
    jif = sample['Journal Impact Factor'].apply(lambda x: float(x) if x!='Not Available' else np.NaN).mean()#.dropna().mean()

    return jif

In [16]:
def get_jif_for_group(group,time_gap = 1,time_window = 1):
    results = []
    for topic in group:
        results.append(Jif_before_changing(topic,time_gap,time_window))
    return [i for i in results if i == i]

In [17]:
def results(time_gap):
    e = get_jif_for_group(emerging_topic_list,time_gap=time_gap)
    s = get_jif_for_group(shrinking_topic_list,time_gap=time_gap)
    return np.mean(e),np.mean(s),ttest_ind(e,s,equal_var=False)[1]/2

In [19]:
results(1)

(3.1001316464214086, 2.0972260985374875, 6.178369829492827e-05)

In [20]:
results(2)

(3.1613007631924455, 2.0002437351966784, 5.650569745786792e-05)

In [21]:
results(3)

(3.790199478513833, 2.3088350701945077, 0.04101364170509293)

In [22]:
results(4)

(2.5337120396819586, 2.132831935879191, 0.07018085840889922)

In [23]:
results(5)

(2.2989876720106843, 2.269502392721279, 0.46548584917789204)